# 导入库

In [ ]:
import os
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from scipy.stats import reciprocal
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import pickle

from Lib import *

devices = tf.config.list_physical_devices()
try:
    tf.config.set_visible_devices(devices[0])
except:
    print('set only CPU failed!')

print('logical_devices: ',tf.config.list_logical_devices())
print('visible_devices: ',tf.config.get_visible_devices())



# 加载所有数据

In [ ]:
loader = DataLoader(train_data_folder='1',test_data_folder='1',forward_steps=5,freq=200,pace=1,valid_size=0.1)
X_train,Y_train,X_valid,Y_valid,X_test,Y_test = loader.load()
print('Train dataset size: ',X_train.shape,Y_train.shape)
print('Valid dataset size: ',X_valid.shape,Y_valid.shape)
print('Test  dataset size: ',X_test.shape,Y_test.shape)

# 建立模型

In [ ]:
def build_model(n_hidden=1,n_neurons_hidden=64,n_neurons=64,activate_func='relu',optimizer_name='Adam',dropout=0.075,lr=0.005):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=[None,12]))
    for layer in range(n_hidden):
        model.add(keras.layers.LSTM(n_neurons_hidden,activation=activate_func,return_sequences=True,input_shape=[None,12]))
    model.add(keras.layers.LSTM(n_neurons,activation=activate_func,input_shape=[None,12]))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(64))
    model.add(keras.layers.Dense(1))
    model.summary()
    if optimizer_name == 'Adam':
        optimizer = keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_name == 'SGD':
        optimizer = keras.optimizers.SGD(learning_rate=lr)
    model.compile(loss="mse", optimizer=optimizer)
    return model

model = build_model(n_hidden=0,n_neurons_hidden=64,n_neurons=128,activate_func='relu',optimizer_name='Adam',dropout=0.075,lr=0.1)

# 训练

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20,restore_best_weights=True)
history = model.fit(X_train,Y_train, 
                    epochs=100,
                    validation_data=(X_valid, Y_valid),
                    callbacks=[early_stopping_cb])

In [ ]:
with open('trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
with open('trainHistoryDict.txt','rb') as file_pi:
    history=pickle.load(file_pi)

In [ ]:
def plot_learning_curves(loss, val_loss):
    plt.plot(np.arange(len(loss)) + 0.5, loss, "b.-", label="Training loss")
    plt.plot(np.arange(len(val_loss)) + 1, val_loss, "r.-", label="Validation loss")
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    #plt.axis([1, 20, 0, 0.05])
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)

plot_learning_curves(history["loss"], history["val_loss"])
#plt.savefig('./train_epoch.jpg')
plt.show()

# 评估训练结果

In [ ]:
model.evaluate(X_test, Y_test)

# 预测输出

In [ ]:
model = keras.models.load_model("./temp_model/test.h5")

In [ ]:
tester = Tester(model=model,freq=200,forward_steps=5,test_data_folder='5')
tester.test()